In [1]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
# import matplotlib.pyplot as plt 
import time
# import numpy as np
# import scipy as sp
import re
from collections import Counter
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC, SVC
# from sklearn.pipeline import Pipeline
# from sklearn import metrics

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [2]:
query = session.query(Article).filter(or_(Article.journal_ref.like('Physics Review Letters%'),
                                          Article.journal_ref.like('Phys. Rev. Lett.%'),
                                          Article.journal_ref.like('PRL%')))
# query = session.query(Article).filter(or_(Article.journal_ref.like('Nature%'),
#                                           Article.journal_ref.like('Nat.%'),
#                                           Article.journal_ref.like('Science%')))
abstract_all = [x.abstract for x in query.all()]
# titleNatureScience = [x.title for x in query.all()]
session.close_all()

In [3]:
abstract_train = abstract_all
print len(abstract_train)

14390


CountVectorizer might do a cleaner job.

In [4]:
class AbstractWriter:
    '''This class will take a specific n-gram and learn to write abstracts from a few words.'''
    _data = dict()
    _abstracts = []
    
    def __init__(self, ngram, randomize=False, seed=None, maxWords=200):
        self.ngram = ngram
        self.randomize = randomize
        self.seed = seed
        self.maxWords = maxWords
        if seed is not None:
            random.seed(seed)

    
    def fit(self, train):
        data_list = []
        for item in train:
            ind = 0
            textlist = self.__cleanup_text(item)
            self._abstracts.append(' '.join(textlist))
            while ind + self.ngram - 1 < len(textlist):
                currentTuple = tuple(textlist[ind:(ind+self.ngram-1)])
                nextWord = textlist[ind+self.ngram-1]
                data_list.append((tuple(textlist[ind:(ind+self.ngram-1)]),
                                        textlist[ind+self.ngram-1]))
                ind += 1
        data_list.sort()
        
        # Now that the list is sorted, convert it into a dictionary.
        # Since the list is sorted, each subsuquent entry is either new or a continuation of the previous one.

        previousKey = None
        for val in data_list:
            currentKey = val[0]
            if currentKey == previousKey:
                self._data[currentKey].append(val[1])
            else:
                self._data[currentKey] = [val[1]]
            previousKey = val[0]

    def find_next_word(self, phrase):
        '''phrase is a tuple of (typ. 3) words. Returns the next word.'''
        # Since data is sorted, it should be possible to search quickly. I'm not doing this properly.
        if phrase in self._data.keys():
            solution = self._data[phrase]
            
            # If there are more than one result, we either return the most popular option or
            # a random answer weighted by popularity.
            if self.randomize:
                return random.choice(solution)
            else:
                count = Counter(solution)
                return count.most_common(1)[0][0]
        else:
            return None
        
    def write_abstract(self, beginning):
        abstract = list(beginning)
        nextWord = ''
        ind = len(beginning) - self.ngram + 1
        while (nextWord is not None) and (ind < self.maxWords):
            nextWord = self.find_next_word(tuple(abstract[ind:(ind+self.ngram)]))
            if nextWord is None:
                break
            else:
                abstract.append(nextWord)
                ind += 1
        return ' '.join(abstract)
    
    def find_similar(self, abstract):
        '''Have I accidentally plagarized one of the articles?
        With too few choices, we will likely just copy an existing article.
        Returns the longest chunk highlighted in an existing article.'''
        first = 0
        last = 1
        abort = False
        longestChunk = ''
        longestChunkLength = 0

        totalLength = len(abstract.split())
        while first + longestChunkLength < totalLength:
            chunk = ' '.join(abstract.split()[first:last])
            if any(x for x in self._abstracts if chunk in x):
                if last - first + 1 > longestChunkLength:
                    longestChunk = chunk
                    longestChunkLength = last - first + 1
                last += 1
            else:
                first += 1
                last = first + 1
        
        best_abstract = (x for x in self._abstracts if longestChunk in x).next()
        return re.sub('(' + longestChunk + ')', '[[[' + longestChunk + ']]]', best_abstract)


    def __cleanup_text(self, text):
        # re from kaggle https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-wordsi
        # Remove punctuation. Keep periods.
        # 
        # New trick: replace numbers with ##, which we will later replace with different numbers.
        pattern1 = re.compile("[0-9]")
        pattern2 = re.compile("[^a-zA-Z#.]")
        # The previous pattern could make multiple spaces. Turn multiple spaces into a single space.
        pattern3 = re.compile("(\s{2,})")
        return pattern3.sub(" ",
                            pattern2.sub(" ",
                                         pattern1.sub("#",
                                                      text.lstrip().rstrip()))).split(' ')

"In this Letter, we" is a popular phrase. Let's make a bunch of abstracts that start that way, and find if they accidentally plagarize anything.

In [16]:
def InThisLetterWe(ngram):
    writer = AbstractWriter(ngram=ngram, randomize=True, seed=42)
    writer.fit(abstract_train)
    basephrase = ['In','this', 'Letter', 'we']
#     if ngram == 6:
#         writer5 = AbstractWriter(ngram=5, randomize=True, seed=42)
#         phrase += random.choice(['report'], ['show'])
    if ngram == 6:
        writer5 = AbstractWriter(ngram=5, randomize=True, seed=42)
    for i in range(0,5):
        if ngram == 6:
            phrase = basephrase + [writer5.find_next_word(tuple(basephrase))]
        else:
            phrase = basephrase
        currentAbstract = writer.write_abstract(phrase)
        print 'New abstract: ' + currentAbstract
        print ''
        print 'Existing abstract: ' + writer.find_similar(currentAbstract)
        print ''
    return writer

def WriteAbstract(ngram):
    writer = AbstractWriter(ngram=ngram, randomize=True, seed=42)
    writer.fit(abstract_train)
    for i in range(0,5):
        phrase = random.choice(writer._abstracts).split()[0:(ngram-1)]
        currentAbstract = writer.write_abstract(tuple(phrase))
        print 'New abstract: ' + currentAbstract
        print ''
        print 'Existing abstract: ' + writer.find_similar(currentAbstract)
        print ''
    return writer

In [6]:
InThisLetterWe(4)

New abstract: In this Letter we report a distinct additional contribution to thermal conductivity. Moreover this low temperature contribution prevails a definite conclusion on the validity or violation of the symmetry breaking is enormously stronger for heterovalent substitution so that the initial state of the incident photons form a pulse train which enters the loops. Dynamically controlled loop coupling ratios allow the construction of a protocol that mimicking what happens to the guests of the hotel maps the amplitudes of transmitted and reflected waves in the interacting two qubit system is probed by extensive quantum Monte Carlo simulations under strong Coulombic couplings. The present results suggest that the quantum metallic phase following the magnetic field we reveal a novel evolution mechanism forcing the LBs to follow varying dispersion diffraction conditions until they leave their existence range and decay.

Existing abstract: We report the first experimental observation o

<__main__.AbstractWriter instance at 0x105fe1c68>

In [7]:
InThisLetterWe(5)

New abstract: In this Letter we report a direct experimental implementation of such a witness in a room temperature microfabricated surface trap without the use of quantum resources. Such benchmarks have been established for special families of input states like coherent states and particular subclasses of squeezed states. Here we present a coherent memory using a pulsed magnetic field gradient and demonstrate the storage and retrieval process of single photon level coherent pulse. This reliable quantum memory is a crucial step toward quantum networks based on solid state devices. The multimode capacity of rare earth based optical processor goes beyond the temporal and the spectral degree of freedom which might provide a useful tool for photonic information processing.

Existing abstract: Here we present the quantum storage of three dimensional orbital angular momentum photonic entanglement in a rare earth ion doped crystal. The properties of the entanglement and the storage process ar

<__main__.AbstractWriter instance at 0x104973638>

In [8]:
InThisLetterWe(6)

New abstract: In this Letter we report a direct experimental implementation of such a witness in a room temperature nuclear magnetic resonance setup which provides a proof of concept demonstration for the usefulness of discord in sensing applications. Discordant probes are shown to guarantee a nonzero precision in the estimation procedure for different generating Hamiltonians while classically correlated probes are unable to accomplish the estimation in a worst case setting. This work establishes a rigorous and direct operational interpretation for general quantum correlations shedding light on their potential for quantum technology.

Existing abstract: Quantum metrology exploits quantum mechanical laws to improve the precision in estimating technologically relevant parameters such as phase frequency or magnetic fields. Probe states are usually tailored on the particular dynamics whose parameters are being estimated. Here we consider a novel framework where quantum estimation is perfor

<__main__.AbstractWriter instance at 0x10d9fd5f0>

In [17]:
WriteAbstract(4)

New abstract: We study the Kondo effect as in heavy fermion compound CeRhIn#. When long range antiferromagnetic correlations. The physics in the proximity of the superconducting gap affect qubits differently from quasiparticles at the impurity which results in a novel waveguide architecture. The TeO# slab waveguide deposited on a W ### single crystal were measured down to very low temperatures although the correlation length xi psi v F Delta psi approx #.# ell # . We speculate that this latter behavior is due to the damping enhancement. Without it a much shorter attosecond timescale.

Existing abstract: We present the first numerical computation of the neutral fermion gap Delta psi in the nu # # quantum Hall state which is analogous to the energy gap for a Bogoliubov de Gennes quasiparticle in a superconductor. We find Delta psi approx #.### frac e # epsilon ell # comparable to the charge gap and discuss the implications for topological quantum information processing. We also deduce an

<__main__.AbstractWriter instance at 0x150353638>

In [18]:
WriteAbstract(5)

New abstract: We analyze the interplay between intra pocket and inter pocket pairing depending on the ellipticity of electron pockets and the strength of its quantum component.

Existing abstract: Pairing symmetry in recently discovered Fe based metallic superconductors AFe # Se # A K Rb Cs with high transition temperature T c sim ## K is currently a subject of intensive debates. These systems contain only electron pockets according to photoemission and differ from the majority of Fe based superconductors in which both electron and hole pockets are present. Both d wave and s wave pairing symmetries have been proposed for AFe # Se # but a d wave gap generally has nodes while experiments clearly point to no nodal behavior and a conventional s wave gap is inconsistent with the observation of the neutron resonance below T c . We argue that current theories of pairing in such systems are incomplete and must include not only intra pocket pairing condensate but also inter pocket condensate ma

<__main__.AbstractWriter instance at 0x185402098>

In [19]:
WriteAbstract(6)

New abstract: Artificial gauge fields open new possibilities to realize quantum many body systems with present technology. Remarkably the emergence of thermal states does not follow from a coupling to an environment but is a result of the physics of the pnictides.

Existing abstract: We establish a setting atoms in optical superlattices with period # in which one can experimentally probe signatures of the process of local relaxation and apparent thermalization in non equilibrium dynamics without the need of addressing single sites. This opens up a way to explore the convergence of subsystems to maximum entropy states in quenched [[[quantum many body systems with present technology. Remarkably the emergence of thermal states does not follow from a coupling to an environment but is a result of the]]] complex non equilibrium dynamics in closed systems. We explore ways of measuring the relevant signatures of thermalization in this analogue quantum simulation of a relaxation process exploit

<__main__.AbstractWriter instance at 0x17564e1b8>

Deterministic: always choose the most popular word.

In [9]:
# 4 grams: take 3 words and predict the 4th
start = time.time()
writer4 = AbstractWriter(ngram=4)
writer4.fit(abstract_train)
print time.time() - start
abstract1a = writer4.write_abstract(['a', 'function', 'of', 'the'])
abstract2a = writer4.write_abstract(['Here','we', 'observe', 'the'])
abstract3a = writer4.write_abstract(['We','measure', 'the', 'mass'])
abstract4a = writer4.write_abstract(['In','this', 'Letter', 'we'])
print abstract1a
print ''
print abstract2a
print ''
print abstract3a
print ''
print abstract4a
print time.time() - start

15.7712628841
a function of the interaction strength and the cavity is suppressed for very flexible membranes. In the less deformable regime we observe a strong reduction of the effective mass of the top quark mass and the strength of the electron spin resonance ESR and decoherence. We propose spin resolved quantum rate equations with correlation functions in Laplace space for the analytical derivation of these results.

Here we observe the formation of a spin # # Heisenberg antiferromagnet on a square lattice with a nearest neighbor interaction and filling. The coexistence of these two quantities shows how our results may be relevant for the design of a novel class of materials with high conversion efficiency. Some strategies have been used to determine the temperature dependence of the spin orbit coupling in the electronic structure of the material and represents the number of atoms in a one dimensional optical lattice. We show that the spin orbit coupling in the electronic structure

In [10]:
# 5 grams: take 4 words and predict the 5th
start = time.time()
writer5 = AbstractWriter(ngram=5)
writer5.fit(abstract_train)
print time.time() - start
abstract1b = writer5.write_abstract(['a', 'function', 'of', 'the'])
abstract2b = writer5.write_abstract(['Here','we', 'observe', 'the'])
abstract3b = writer5.write_abstract(['We','measure', 'the', 'mass'])
abstract4b = writer5.write_abstract(['In','this', 'Letter', 'we'])
print abstract1b
print ''
print abstract2b
print ''
print abstract3b
print ''
print abstract4b
print time.time() - start

17.3439519405
a function of the interaction strength and temperature. At low temperature thermal fluctuations induce magnetic long range order of the Eu #f moments.

Here we observe the phenomenon of sudden transition of quantum correlations when local phase and amplitude damping channels are applied to the case of a single impurity immersed in a Fermi sea close to an interspecies p wave Feshbach resonance. We show that the spin orbit interaction and the crossing of the critical point. We also make a quantitative prediction on increasing the upper critical field B c# T of the superconductor. We predict an enhancement of the shot noise in the tubes. In contrast to Coulomb drag of the average current the noise locking is based on a data sample corresponding to an integrated luminosity of #.# fb # collected with the LHCb detector at a center of mass energy of # TeV collected by the CDF II detector at the Fermilab Tevatron Collider. We derive ## CL upper limits on the branching fractions a

In [11]:
# 6 grams: take 5 words and predict the 6th
start = time.time()
writer6 = AbstractWriter(ngram=6, randomize=True, seed=42)
writer6.fit(abstract_train)
print time.time() - start
abstract1c = writer6.write_abstract(['a', 'function', 'of', 'the', 'interaction'])
abstract2c = writer6.write_abstract(['Here','we', 'observe', 'the', 'phenomenon'])
abstract3c = writer6.write_abstract(['We','measure', 'the', 'mass', 'of'])
abstract4c = writer6.write_abstract(['In','this', 'Letter', 'we', 'show'])
print abstract1c
print ''
print abstract2c
print ''
print abstract3c
print ''
print abstract4c
print time.time() - start

17.2727239132
a function of the interaction strength and other relevant properties characterizing the Fermi liquid behavior effective mass discontinuity at the Fermi surface and pair correlation function. In the high density regime the anisotropic effect plays an important role. We find phase transitions from singlet pairing to FFLO and further to S z # triplet pairing can generally take place upon increasing the magnetic field when strong charge fluctuations coexist with spin fluctuations and ii the enhancement of the charge fluctuations lead to a significant increase of the pairing correlations and critical temperature relative to the homogeneous system when the modulation length scale is sufficiently large. With a separable form of the irreducible particle particle vertex we show that optimized superconductivity is obtained for moderate modulation strength due to a delicate balance between the number of high degree nodes in the network and the topological distance between them dicta

Randomized approach. The next word is weighted by its frequency in previous documents.

In [12]:
# 4 grams: take 3 words and predict the 4th
start = time.time()
writer4rand = AbstractWriter(ngram=4, randomize=True, seed=42)
writer4rand.fit(abstract_train)
print time.time() - start
abstract1a = writer4rand.write_abstract(['a', 'function', 'of', 'the'])
abstract2a = writer4rand.write_abstract(['Here','we', 'observe', 'the'])
abstract3a = writer4rand.write_abstract(['We','measure', 'the', 'mass'])
abstract4a = writer4rand.write_abstract(['In','this', 'Letter', 'we'])
print abstract1a
print ''
print abstract2a
print ''
print abstract3a
print ''
print abstract4a
print time.time() - start

17.1299350262
a function of the noise affecting the full set of nonlinear equations that involve arbitrary dephasing Hamiltonian operators.

Here we observe the lattice induced phase shifts for inferring the variation of inclusive jet production are performed using the information obtained from the former in the absence of gapped states of the corner opening angle and was recently proposed as a mechanism for significantly enhancing the fidelity. Our scheme is particularly useful in the low energy Fermi liquid fixed point in the disordered phase. Analogies to the feedback effect in unconventional heavy fermion superconductors U # x Th x Be ## and UPt # we speculate that the small Pr behavior is due to the antibonding Cu O related band while the hump is mainly formed by its bonding counterpart with a c axis bilayer coupling induced splitting of about ### nm at the ODT. We argue that current neutron star observations exclude asymmetric bosonic non interacting dark matter models. We establ

In [13]:
# 5 grams: take 4 words and predict the 5th
start = time.time()
writer5rand = AbstractWriter(ngram=5, randomize=True, seed=42)
writer5rand.fit(abstract_train)
print time.time() - start
abstract1b = writer5rand.write_abstract(['a', 'function', 'of', 'the'])
abstract2b = writer5rand.write_abstract(['Here','we', 'observe', 'the'])
abstract3b = writer5rand.write_abstract(['We','measure', 'the', 'mass'])
abstract4b = writer5rand.write_abstract(['In','this', 'Letter', 'we'])
print abstract1b
print ''
print abstract2b
print ''
print abstract3b
print ''
print abstract4b
print time.time() - start

31.7281160355
a function of the number of data points provided i.e. given a large enough data set two point clouds can always be recognized as different clusters as long as their separation is non zero. In contrast for the sparse networks studied here a cluster structure remains undetectable even in an infinitely large network if a critical separation is not exceeded. We give analytic formulas for this critical separation as a function of interdot distance. Slight mismatch introduced in the fabrication of epitaxial superconducting LaFeAs O F thin films we performed an extensive study of bismuth s infrared conductivity under pressure. In contrast to the #D hybridized thin film an additional weakly field dependent longitudinal resistance is observed in the temperature dependence of the TEP shows a change of slope at T ### K and the Kohler rule breaks down above ## ### K range. The Fermi surface consists of six disconnected segments consistent with recent theoretical calculations of excit

In [14]:
# 6 grams: take 5 words and predict the 6th
start = time.time()
writer6rand = AbstractWriter(ngram=6, randomize=True, seed=42)
writer6rand.fit(abstract_train)
print time.time() - start
abstract1c = writer6rand.write_abstract(['a', 'function', 'of', 'the', 'interaction'])
abstract2c = writer6rand.write_abstract(['Here','we', 'observe', 'the', 'phenomenon'])
abstract3c = writer6rand.write_abstract(['We','measure', 'the', 'mass', 'of'])
abstract4c = writer6rand.write_abstract(['In','this', 'Letter', 'we', 'show'])
print abstract1c
print ''
print abstract2c
print ''
print abstract3c
print ''
print abstract4c
print time.time() - start

56.1883370876
a function of the interaction strength and other relevant properties characterizing the Fermi liquid behavior effective mass discontinuity at the Fermi surface and pair correlation function. In the high density regime the anisotropic effect plays an important role. We find phase transitions from singlet pairing to FFLO and further to S z # triplet pairing can generally take place upon increasing the magnetic field when strong charge fluctuations coexist with spin fluctuations and ii the enhancement of the charge fluctuations lead to a significant increase of the pairing correlations and critical temperature relative to the homogeneous system when the modulation length scale is sufficiently large. With a separable form of the irreducible particle particle vertex we show that optimized superconductivity is obtained for moderate modulation strength due to a delicate balance between the number of high degree nodes in the network and the topological distance between them dicta

In [15]:
# for item in abstract_train:
#     if 'we measure the mass of the' in item:
#         print item
# print ''

# for item in abstract_train:
#     if 'in this letter we show that the spin' in item.lower():
#         print item
# print ''

# for item in abstract_train:
#     if 'a function of the interaction' in item.lower():
#         print item
# print ''

# for item in abstract_train:
#     if 'here we observe the phenomenon' in item.lower():
#         print item
# print ''

# for item in abstract_train:
#     if 'leads to the emergence' in item.lower():
#         print item
# print ''



What does the data tell us about entropy of abstracts? See "Prediction and Entropy of Printed English" by C. Shannon, http://languagelog.ldc.upenn.edu/myl/Shannon1950.pdf 